Steps to setup a Project

1. Create Virtual Environment and swith to virtual environment
2. pip install uv
3. uv init
4. uv add ipykernel, langchain, langchain-community, pypdf, pymupdf

In [1]:
from langchain_community.document_loaders import (
    PyPDFLoader,
    PyMuPDFLoader
)

In [ ]:
# Method 1 - PypdfLoader
print("PyPdfloader")

try:
    pypdf_loader=PyPDFLoader("attention.pdf")
    pypdf_docs=pypdf_loader.load()
    print(pypdf_docs)
    print(f"  Loaded {len(pypdf_docs)} pages")
    print(f"  Page 1 content: {pypdf_docs[0].page_content[:100]}...")
    print(f"  Metadata: {pypdf_docs[0].metadata}")

except Exception as e:
    print(f"Error : {e}")

In [ ]:
# Method 2: PyMuPDFLoader (Fast and accurate)
print("\n3️⃣ PyMuPDFLoader")
try:
    pymupdf_loader = PyMuPDFLoader("attention.pdf")
    pymupdf_docs = pymupdf_loader.load()
    
    print(f"  Loaded {len(pymupdf_docs)} pages")
    print(f"  Includes detailed metadata")
    print(pymupdf_docs)
except Exception as e:
    print(f"  Error: {e}")

# 📊 PDF Loader Comparison
print("\n📊 PDF Loader Comparison:")
print("\nPyPDFLoader:")
print("  ✅ Simple and reliable")
print("  ✅ Good for most PDFs")
print("  ✅ Preserves page numbers")
print("  ❌ Basic text extraction")
print("  Use when: Standard text PDFs")

print("\nPyMuPDFLoader:")
print("  ✅ Fast processing")
print("  ✅ Good text extraction")
print("  ✅ Image extraction support")
print("  Use when: Speed is important")

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List
class SmartPDFProcessor:
    """Advanced PDF processing with error handling"""
    def __init__(self,chunk_size=1000,chunk_overlap=100):
        self.chunk_size=chunk_size,
        self.chunk_overlap=chunk_overlap,
        self.text_splitter=RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=[" "],

        )

    def process_pdf(self,pdf_path:str)->List[Document]:
        """Process PDF with smart chunking and metadata enhancement"""

        # Laod PDF

        loader=PyPDFLoader(pdf_path)
        pages=loader.load()

        ## Process each page

        processed_chunks=[]

        for page_num,page in enumerate(pages):
            ## clean text
            cleaned_text=self._clean_text(page.page_content)

            # Skip nearly empty pages
            if len(cleaned_text.strip()) < 50:
                continue

            # Create chunks with enhanced metadata
            chunks = self.text_splitter.create_documents(
                texts=[cleaned_text],
                metadatas=[{
                    **page.metadata,
                    "page": page_num + 1,
                    "total_pages": len(pages),
                    "chunk_method": "smart_pdf_processor",
                    "char_count": len(cleaned_text)
                }]
            )
            
            processed_chunks.extend(chunks)

        return processed_chunks

    def _clean_text(self, text: str) -> str:
        """Clean extracted text"""
        # Remove excessive whitespace
        text = " ".join(text.split())
        
        # Fix common PDF extraction issues
        text = text.replace("ﬁ", "fi")
        text = text.replace("ﬂ", "fl")
        
        return text

In [ ]:
## Process a PDF if available
try:
    preprocessor=SmartPDFProcessor()
    smart_chunks=preprocessor.process_pdf("attention.pdf")
    print(f"Processed into {len(smart_chunks)} smart chunks")

    # Show enhanced metadata
    if smart_chunks:
        print("\nSample chunk metadata:")
        for key, value in smart_chunks[0].metadata.items():
            print(f"  {key}: {value}")

except Exception as e:
    print(f"Processing error: {e}")